In [1]:
from datasets import load_dataset
from datasets.arrow_dataset import Dataset
from datasets.dataset_dict import DatasetDict, IterableDatasetDict
from datasets.iterable_dataset import IterableDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset as HFDataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

/home/nova/anaconda3/envs/deci/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_set = pd.read_csv("spam.tsv",sep='\t')

In [3]:
data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    5572 non-null   object
 1   message  5572 non-null   object
 2   length   5572 non-null   int64 
 3   punct    5572 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 174.2+ KB


In [4]:
data_set

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,160,8
5568,ham,Will ü b going to esplanade fr home?,36,1
5569,ham,"Pity, * was in mood for that. So...any other s...",57,7
5570,ham,The guy did some bitching but I acted like i'd...,125,1


In [5]:
label_map = {"ham": 0, "spam": 1}
data_set["label"] = data_set["label"].map(label_map)

In [6]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data_set["message"], data_set["label"], test_size=0.2, random_state=42
)

In [7]:
model_id = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [8]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=512)

In [9]:
train_dataset = HFDataset.from_dict({"input_ids": train_encodings["input_ids"], 
                                     "attention_mask": train_encodings["attention_mask"], 
                                     "labels": list(train_labels)})

val_dataset = HFDataset.from_dict({"input_ids": val_encodings["input_ids"], 
                                   "attention_mask": val_encodings["attention_mask"], 
                                   "labels": list(val_labels)})

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    push_to_hub=False,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

/home/nova/anaconda3/envs/deci/lib/python3.10/site-packages/transformers/training_args.py:1573: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/1674 [00:00<?, ?it/s]/home/nova/anaconda3/envs/deci/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:167: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
  1%|          | 10/1674 [00:13<16:50,  1.65it/s] 

{'loss': 0.3824, 'grad_norm': 7.111941814422607, 'learning_rate': 1.9880525686977302e-05, 'epoch': 0.02}


  1%|          | 20/1674 [00:18<13:53,  1.98it/s]

{'loss': 0.1707, 'grad_norm': 4.712263107299805, 'learning_rate': 1.97610513739546e-05, 'epoch': 0.04}


  2%|▏         | 30/1674 [00:23<13:41,  2.00it/s]

...

                                                   
100%|██████████| 1674/1674 [11:41<00:00,  3.35it/s]

{'eval_loss': 0.06016705557703972, 'eval_runtime': 7.7486, 'eval_samples_per_second': 143.897, 'eval_steps_per_second': 18.068, 'epoch': 3.0}


100%|██████████| 1674/1674 [11:43<00:00,  2.38it/s]

{'train_runtime': 703.3076, 'train_samples_per_second': 19.012, 'train_steps_per_second': 2.38, 'train_loss': 0.029778581343443626, 'epoch': 3.0}


TrainOutput(global_step=1674, training_loss=0.029778581343443626, metrics={'train_runtime': 703.3076, 'train_samples_per_second': 19.012, 'train_steps_per_second': 2.38, 'total_flos': 2340429376636764.0, 'train_loss': 0.029778581343443626, 'epoch': 3.0})

In [20]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Should be > 0 if GPU is available
print(torch.cuda.current_device())  # Should return a valid device index
print(torch.cuda.get_device_name(0))  # Prints your GPU name


True
1
0
NVIDIA GeForce RTX 4060 Laptop GPU


In [12]:
model.save_pretrained("./fine_tuned_spam_model")
tokenizer.save_pretrained("./fine_tuned_spam_model")

print("Fine-tuning complete. Model saved.")

Fine-tuning complete. Model saved.


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Path to your fine-tuned model
fine_tuned_model_path = "./fine_tuned_spam_model"  # Change this to the correct path

# Load the tokenizer and fine-tuned model
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path)

# Set the model to evaluation mode
model.eval()


/home/nova/anaconda3/envs/deci/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModernBertForSequenceClassification(
  (model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(50368, 768, padding_idx=50283)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertRotaryEmbedding()
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=2304, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=1152, out_features=768, bias=False)
        )
      

In [6]:
def classify_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()

    label_map = {0: "ham", 1: "spam"}
    return label_map[prediction]

sentence = """we are calling from union bank of india the credit card declined last week
we would help you to redeem the card and give you free expensive gifts aling with it so please give the OTP to redeem the gift"""
print(f"Predicted Label: {classify_text(sentence)}")



Predicted Label: spam
